In [ ]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.o

In [ ]:
!pip uninstall jaxlib
#!pip install jaxlib==0.4.2
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.3.25+cuda11.cudnn805.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 MB 5.3 MB/s eta 0:00:00


In [1]:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
#%env JAX_PLATFORMS=cpu
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [3]:
from utils import *
from layers import two_wl_aggregation

In [4]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
import shutil
import os

In [5]:
# for input "kernel_fn(x1, x2, 'ntk', pattern = (p1,p2))"
# the shape of the kernel after the convolution is:
# x1 = (b1, h1, w1, c1)
# x2 = (b2, h2, w2, c2)
# k = (b1, b2, h1, h2, w1, w2) 


L_branche = stax.serial(
    stax.Conv(100, (1,1)),
    )

Gamma_branche = stax.serial(
    stax.Conv(100, (1,1)),
    two_wl_aggregation(n_nodes=28),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
    )


init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer, # k = (b1, b2, h1, h2, w1, w2)
    stax.GlobalSumPool(), # # k = (b1, b2)
)




In [6]:
#this needs ~9GB gpu-ram, takes ~15s on GPU ~2min on CPU (size 4 using XLA_PYTHON_CLIENT_PREALLOCATE=false)
%%time 
size = 10
r = 18
i = 0
j = 0

path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/splits/size_{size}_range_{r}"

x1 = np.load(path + f"/graphs_edge_features_1_{i*size}_{(1+i)*size}.npy")
x2 = np.load(path + f"/graphs_edge_features_2_{j*size}_{(1+j)*size}.npy")
p1 = np.load(path + f"/two_wl_pattern_radius_1_1_{i*size}_{(1+i)*size}.npy")
p2 = np.load(path + f"/two_wl_pattern_radius_1_2_{j*size}_{(1+j)*size}.npy")

kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))

/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


CPU times: user 10 s, sys: 1.95 s, total: 12 s
Wall time: 17.3 s


In [7]:
dataset_size = 188
size = 10
r = 18

base_path = f"/content/drive/MyDrive/MasterarbeitData"
data_path = base_path + f"/Preprocessed/MUTAG"
split_data_path = data_path + f"/splits/size_{size}_range_{r}" 
kernel_path = base_path + f"/Kernels/MUTAG/TWO_WL"
if not os.path.exists(kernel_path):
    os.makedirs(kernel_path)

In [8]:
# clear the kernel folder
# shutil.rmtree(kernel_path)

In [9]:
# at some point this allways fails with 15gb gpu memory
# fails always for some function in utils
# can we make those function more memory efficient?

# with XLA_PYTHON_CLIENT_ALLOCATOR=platform this seems to stay below 5GB 
# GPU-RAM usage for size=4

# calculate kernel from saved preprocessed data
for i in range(r+1):
  for j in range(r+1):

    l = lambda x : x*size 
    h = lambda x : (1+x)*size if (not x == r) else dataset_size-1

    x1 = np.load(split_data_path + f"/graphs_edge_features_1_{l(i)}_{h(i)}.npy")
    x2 = np.load(split_data_path + f"/graphs_edge_features_2_{l(j)}_{h(j)}.npy")
    p1 = np.load(split_data_path + f"/two_wl_pattern_radius_1_1_{l(i)}_{h(i)}.npy")
    p2 = np.load(split_data_path + f"/two_wl_pattern_radius_1_2_{l(j)}_{h(j)}.npy")

    print(p1.shape)
    print(p2.shape)

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(kernel_path + f"/NTK_{h(i)}_{h(j)}", kernel_matrix)

(888, 4)
(888, 4)
(888, 4)
(948, 4)
(888, 4)
(1140, 4)
(888, 4)
(904, 4)
(888, 4)
(964, 4)
(888, 4)
(1164, 4)
(888, 4)
(960, 4)
(888, 4)
(954, 4)
(888, 4)
(956, 4)
(888, 4)
(1072, 4)
(888, 4)
(1276, 4)
(888, 4)
(800, 4)
(888, 4)
(922, 4)
(888, 4)
(880, 4)
(888, 4)
(744, 4)
(888, 4)
(870, 4)
(888, 4)
(1096, 4)
(888, 4)
(1054, 4)
(888, 4)
(618, 4)
(948, 4)
(888, 4)
(948, 4)
(948, 4)
(948, 4)
(1140, 4)
(948, 4)
(904, 4)
(948, 4)
(964, 4)
(948, 4)
(1164, 4)
(948, 4)
(960, 4)
(948, 4)
(954, 4)
(948, 4)
(956, 4)
(948, 4)
(1072, 4)
(948, 4)
(1276, 4)
(948, 4)
(800, 4)
(948, 4)
(922, 4)
(948, 4)
(880, 4)
(948, 4)
(744, 4)
(948, 4)
(870, 4)
(948, 4)
(1096, 4)
(948, 4)
(1054, 4)
(948, 4)
(618, 4)
(1140, 4)
(888, 4)
(1140, 4)
(948, 4)
(1140, 4)
(1140, 4)
(1140, 4)
(904, 4)
(1140, 4)
(964, 4)
(1140, 4)
(1164, 4)
(1140, 4)
(960, 4)
(1140, 4)
(954, 4)
(1140, 4)
(956, 4)
(1140, 4)
(1072, 4)
(1140, 4)
(1276, 4)
(1140, 4)
(800, 4)
(1140, 4)
(922, 4)
(1140, 4)
(880, 4)
(1140, 4)
(744, 4)
(1140, 4)
(870,

/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'
